In [3]:
df = pd.read_csv('secondhanddataset.csv')
df

,v.id,on road old,on road now,years,km,rating,condition,economy,top speed,hp,torque,current price
0,1,535651,798186,3,78945,1,2,14,177,73,123,351318.0
1,2,591911,861056,6,117220,5,9,9,148,74,95,285001.5
2,3,686990,770762,2,132538,2,8,15,181,53,97,215386.0
3,4,573999,722381,4,101065,4,3,11,197,54,116,244295.5
4,5,691388,811335,6,61559,3,9,12,160,53,105,531114.5
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,633238,743850,5,125092,1,6,11,171,95,97,190744.0
996,997,599626,848195,4,83370,2,9,14,161,101,120,419748.0
997,998,646344,842733,7,86722,1,8,9,196,113,89,405871.0
998,999,535559,732439,2,140478,4,5,9,184,112,128,74398.0


In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

In [38]:
data = pd.read_csv('secondhanddataset.csv')
X = data[['on road old', 'on road now', 'years', 'km', 'rating', 'condition', 'economy', 'top speed', 'hp', 'torque']]
y = data['current price']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(np.array(y_train).reshape(-1, 1)).flatten()
y_test = scaler_y.transform(np.array(y_test).reshape(-1, 1)).flatten()

In [42]:
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_torch = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

dataset = TensorDataset(X_train_torch, y_train_torch)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [43]:
class MLP_PyTorch(nn.Module):
    def __init__(self):
        super(MLP_PyTorch, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(10, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

In [44]:
model = MLP_PyTorch()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

epochs = 500
for epoch in range(epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

In [45]:
y_pred_torch = scaler_y.inverse_transform(model(X_test_torch).detach().numpy())
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1))
rmse_torch = np.sqrt(mean_squared_error(y_test_original, y_pred_torch))
mse_torch = mean_squared_error(y_test_original, y_pred_torch)
r2_torch = r2_score(y_test_original, y_pred_torch)

In [46]:
model_tf = keras.Sequential([
    keras.layers.Input(shape=(10,)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

model_tf.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00005), loss='mse')

In [47]:
model_tf.fit(X_train, y_train, epochs=500, batch_size=64, verbose=0)

In [48]:
y_pred_tf = scaler_y.inverse_transform(model_tf.predict(X_test))
rmse_tf = np.sqrt(mean_squared_error(y_test_original, y_pred_tf))
mse_tf = mean_squared_error(y_test_original, y_pred_tf)
r2_tf = r2_score(y_test_original, y_pred_tf)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [50]:
print("PyTorch Model: RMSE:", rmse_torch, "MSE:", mse_torch, "R2:", r2_torch)
print("TensorFlow Model: RMSE:", rmse_tf, "MSE:", mse_tf, "R2:", r2_tf)

PyTorch Model: RMSE: 3675.297056117052 MSE: 13507808.450702667 R2: 0.9991964885952587
TensorFlow Model: RMSE: 7792.04027194436 MSE: 60715891.59960274 R2: 0.9963883177994886


**Multi-Layer Perceptron (MLP) untuk Prediksi Harga Mobil Bekas**

1. Apa Itu Multi-Layer Perceptron (MLP)?
Multi-Layer Perceptron (MLP) adalah jenis jaringan saraf tiruan (Artificial Neural Network) yang terdiri dari beberapa lapisan neuron. MLP sering digunakan untuk tugas regresi dan klasifikasi.  

Struktur MLP terdiri dari:
1. Input Layer → Menerima fitur dari dataset.  
2. Hidden Layers → Melakukan perhitungan dan transformasi data menggunakan bobot dan fungsi aktivasi.  
3. Output Layer → Menghasilkan prediksi akhir.  

MLP belajar dengan menyesuaikan bobot melalui backpropagation menggunakan algoritma optimasi seperti Adam. 

Kode ini menggunakan dua model berbasis Multi-Layer Perceptron (MLP) untuk memprediksi harga mobil bekas:

Model dengan PyTorch
Model dengan TensorFlow (Keras)
Keduanya memiliki struktur yang mirip dan digunakan untuk tugas regresi, yaitu memprediksi harga berdasarkan fitur-fitur mobil.

---

2. Arsitektur Model TensorFlow dalam Kode
Model TensorFlow yang digunakan memiliki struktur berikut:
- Input Layer: 10 neuron (sesuai jumlah fitur)
- Hidden Layers:

512 neuron → 256 neuron → 128 neuron → 64 neuron → 32 neuron
Menggunakan ReLU activation function di setiap hidden layer.
- Output Layer: 1 neuron (karena tugas regresi)
- Optimizer: Adam dengan learning rate = 0.00005
- Loss Function: Mean Squared Error (MSE)

Kenapa ReLU?
- Mengatasi masalah vanishing gradient yang sering terjadi pada sigmoid/tanh.  
- Cepat dan sederhana, membantu jaringan belajar lebih cepat.  

---

3. Fungsi Aktivasi yang Digunakan  
ReLU (Rectified Linear Unit):  
\[
f(x) = \max(0, x)
\]
Digunakan di hidden layers untuk memperkenalkan non-linearitas ke dalam model.  

---

4. Optimasi Model  
- Loss Function: Mean Squared Error (MSE)  
  \[
  MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
  \]  
  Digunakan karena ini adalah tugas regresi, dan MSE menghitung selisih kuadrat antara nilai sebenarnya dan prediksi.  

- Optimizer: Adam  
  \[
  m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t
  \]
  \[
  v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2
  \]
  Adam menggabungkan keunggulan Momentum dan RMSprop untuk update bobot yang lebih stabil dan cepat.  

---

5. Evaluasi Model 
Model dievaluasi dengan 3 metrik utama:  
1. Root Mean Squared Error (RMSE)  
   \[
   RMSE = \sqrt{MSE}
   \]
   → Mengukur seberapa besar error dalam satuan asli target. Semakin kecil, semakin baik.  

2. Mean Squared Error (MSE) 
   \[
   MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
   \]
   → Menunjukkan rata-rata error kuadrat.  

3. R-Squared (R² Score)  
   \[
   R^2 = 1 - \frac{SS_{res}}{SS_{tot}}
   \]
   → Mengukur seberapa baik model menjelaskan variasi data. Semakin mendekati 1, semakin baik.

6. Perbandingan Hasil Evaluasi
Metrik	PyTorch Model	TensorFlow Model
RMSE (√MSE)	3,675	7,792
MSE	13,507,808	60,715,891
R² Score	0.9992	0.9964

- Kesimpulan Awal:

Model PyTorch lebih unggul karena memiliki RMSE & MSE lebih kecil dan R² lebih tinggi.
Model TensorFlow masih cukup baik (R² mendekati 1), tetapi performanya lebih buruk dibanding PyTorch.

---

Kesimpulan
Model PyTorch memiliki performa lebih baik dibandingkan model TensorFlow, ditunjukkan oleh nilai RMSE dan MSE yang lebih rendah serta R² yang lebih mendekati 1. Model PyTorch lebih akurat karena memiliki hidden layer awal lebih besar (1024 vs. 512 pada TensorFlow), serta learning rate yang lebih kecil (0.00001 vs. 0.00005), sehingga dapat menangkap lebih banyak pola dalam data tanpa overfitting. Selain itu, kemungkinan perbedaan dalam implementasi optimizer Adam antara PyTorch dan TensorFlow juga turut memengaruhi hasil. Dark hasilnya tentu perlu diperbaiki lagi agar RMSE dan MSE semakin mendekati nol serta model semakin optimal dalam melakukan prediksi.